In [ ]:
import numpy as np
import pandas as p
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
documents_categories = p.read_csv("/Users/astrachan/Desktop/outbrain/documents_categories.csv")
#clicks_test = p.read_csv("/Users/astrachan/Desktop/outbrain/clicks_test.csv")
#documents_meta = p.read_csv("/Users/astrachan/Desktop/outbrain/documents_meta.csv")
#documents_entities = p.read_csv("/Users/astrachan/Desktop/outbrain/documents_entities.csv")
promoted_content = p.read_csv("/Users/astrachan/Desktop/outbrain/promoted_content.csv")
#sample_submission = p.read_csv("/Users/astrachan/Desktop/outbrain/sample_submission.csv")
documents_topics = p.read_csv("/Users/astrachan/Desktop/outbrain/documents_topics.csv")
clicks_train = p.read_csv("/Users/astrachan/Desktop/outbrain/clicks_train.csv", dtype={"display_id": int, "ad_id": int, "clicked" : bool})
events = p.read_csv("/Users/astrachan/Desktop/outbrain/events.csv", dtype={"display_id": int, "uuid": str, "document_id" : int, "timestamp" : int, "platform" : object, "geo_location" : str})
#page_views = p.read_csv("/Users/astrachan/Desktop/outbrain/page_views.csv")
#page_views_sample = p.read_csv("/Users/astrachan/Desktop/outbrain/page_views_sample.csv")

In [ ]:
#add document to clicks_train
events.drop(events.columns[[1,3,4,5]],axis = 1,inplace=True)
clicks_train = clicks_train.merge(events, how='left', on='display_id')
del events

In [ ]:
#split to validation and train
leng = len(clicks_train.display_id.unique())
start_display_id_validation = (4 * leng / 5)
start_index_validation = clicks_train[clicks_train['display_id'] == start_display_id_validation].index.tolist()[0]
validation = clicks_train[start_index_validation:]
train = clicks_train[:start_index_validation]
del clicks_train

In [196]:
#remove all confidence levels below 0.3, can decide on another criteria later on
#documents_entities = documents_entities[documents_entities.confidence_level > 0.3]
documents_categories = documents_categories[documents_categories.confidence_level > 0.3]
documents_topics = documents_topics[documents_topics.confidence_level > 0.3]

In [197]:
#drop confidence level
documents_categories.drop('confidence_level',axis=1, inplace=True)
documents_topics.drop('confidence_level',axis=1, inplace=True)

In [198]:
#prepare one-hot
topic_dummies = p.get_dummies(documents_topics['topic_id'],prefix="_ti")
categories_dummies = p.get_dummies(documents_categories['category_id'],prefix="_ci")

In [199]:
topics = documents_topics.drop('topic_id',1).join(topic_dummies)
categories = documents_categories.drop('category_id',1).join(categories_dummies)
categories = categories.groupby(['document_id']).sum().reset_index()
topics = topics.groupby(['document_id']).sum().reset_index()
del documents_categories, documents_topics, categories_dummies, topic_dummies

In [204]:
#create promoted, promoted content (ad information) with its' categories and topics
promoted_content.drop(promoted_content.columns[[2,3]],axis = 1,inplace=True)
promoted = promoted_content.merge(categories, how='left', on='document_id')
promoted = promoted.merge(topics, how='left', on='document_id').drop('document_id', axis=1)
del promoted_content
#all Nulls are of documents with no topic or category, equals to 0
promoted.fillna(0.0, inplace=True)

In [ ]:
predictors=[x for x in train.columns if x not in ['display_id','ad_id','document_id']]
alg = RandomForestClassifier(random_state=1, n_estimators=3, min_samples_split=4, min_samples_leaf=2, warm_start=True)
for chunk in np.array_split(train, train.shape[0]/10**6):
    chunk = chunk.merge(promoted,how = 'left',on = 'ad_id')
    chunk = chunk.merge(categories, how='left', on='document_id',suffixes=('_ad', '_doc'))
    chunk = chunk.merge(topics, how='left', on='document_id',suffixes=('_ad', '_doc'))
    chunk.fillna(0.0, inplace=True)
    alg.fit(chunk[predictors], chunk["clicked"])

/Users/astrachan/anaconda/lib/python2.7/site-packages/sklearn/ensemble/forest.py:303: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn("Warm-start fitting without increasing n_estimators does not "


In [ ]:
predY=[]
for chunk in np.array_split(validation, validation.shape[0]/10**6):
    init_chunk_size=len(chunk)
    chunk.merge(promoted,how = 'left',on = 'ad_id',inplace=True)
    chunk.merge(categories, how='left', on='document_id',suffixes=('_ad', '_doc'),inplace=True)
    chunk.merge(topics, how='left', on='document_id',suffixes=('_ad', '_doc'),inplace=True)
    chunk.fillna(0.0, inplace=True)
    chunk_pred=list(alg.predict_proba(chunk[predictors]).astype(float)[:,1])
    predY += chunk_pred

In [186]:
start = 0 * chunk_size
end = (0+1) * chunk_size
train_chunk = train[start:end].merge(promoted,how = 'left',on = 'ad_id')
train_chunk = train_chunk.merge(categories, how='left', on='document_id',suffixes=('_ad', '_doc'))
train_chunk = train_chunk.merge(topics, how='left', on='document_id',suffixes=('_ad', '_doc'))
train_chunk.fillna(0.0, inplace=True)
predictors=[x for x in train_chunk.columns if x not in ['display_id','ad_id','document_id']]
alg = RandomForestClassifier(random_state=1, n_estimators=3, min_samples_split=4, min_samples_leaf=2, warm_start=True)
alg.fit(train_chunk[predictors], train_chunk["clicked"])